# Configure CFE using NGEN-CAL Pydantic Classes

In [8]:
import json
from pathlib import Path

In [9]:
from ngen.config.init_config import cfe as cfe_init
from ngen.config import formulation, cfe, sloth, multi, configurations

## Function for parsing CFE parameters from noahowp attributes


def parse_cfe_parameters(
    cfe_noahowp_attributes: pandas.DataFrame,
) -> typing.Dict[str, dict]:
    """
    Parses parameters from NOAHOWP_CFE DataFrame

    Parameters
    ----------
    cfe_noahowp_attributes: pandas.DataFrame
        Dataframe of NoahOWP CFE parameters

    Returns
    -------
    Dict[str, dict]: parsed CFE parameters


In [10]:
from collections import OrderedDict
import pandas
import typing

In [4]:
def parse_cfe_parameters(
    cfe_noahowp_attributes: pandas.DataFrame,
) -> typing.Dict[str, dict]:
    """
    Parses parameters from NOAHOWP_CFE DataFrame

    Parameters
    ----------
    cfe_noahowp_attributes: pandas.DataFrame
        Dataframe of NoahOWP CFE parameters

    Returns
    -------
    Dict[str, dict]: parsed CFE parameters

    """
    catchment_configs = {}
    for idx, row in cfe_noahowp_attributes.iterrows():
        d = OrderedDict()

        # static parameters
        d["forcing_file"] = "BMI"
        d["surface_partitioning_scheme"] = "Schaake"

        # ----------------
        # State Parameters
        # ----------------

        # soil depth
        d["soil_params.depth"] = "2.0[m]"

        # many of these values are taken from the 2m depth in hydrofabrics cfe_noahowp_attributes
        d[
            "soil_params.b"
        ] = f'{row["bexp_soil_layers_stag=2"]}[]'  # 	beta exponent on Clapp-Hornberger (1978) soil water relations

        # saturated hydraulic conductivity
        d["soil_params.satdk"] = f'{row["dksat_soil_layers_stag=2"]}[m s-1]'

        # saturated capillary head
        d["soil_params.satpsi"] = f'{row["psisat_soil_layers_stag=2"]}[m]'

        # this factor (0-1) modifies the gradient of the hydraulic head at the soil bottom. 0=no-flow.
        d["soil_params.slop"] = f'{row["slope"]}[m/m]'

        # saturated soil moisture content
        d["soil_params.smcmax"] = f'{row["smcmax_soil_layers_stag=2"]}[m/m]'

        # wilting point soil moisture content
        d["soil_params.wltsmc"] = f'{row["smcwlt_soil_layers_stag=2"]}[m/m]'

        # ---------------------
        # Adjustable Parameters
        # ---------------------

        # optional; defaults to 1.0
        d["soil_params.expon"] = (
            f'{row["gw_Expon"]}[]' if row["gw_Expon"] is not None else "1.0[]"
        )

        # optional; defaults to 1.0
        # not sure if this is the correct key
        d["soil_params.expon_secondary"] = (
            f'{row["gw_Coeff"]}[]' if row["gw_Coeff"] is not None else "1.0[]"
        )

        # maximum storage in the conceptual reservoir
        d["max_gw_storage"] = (
            f'{row["gw_Zmax"]}[m]' if row["gw_Zmax"] is not None else "0.011[m]"
        )

        # primary outlet coefficient
        d["Cgw"] = "0.0018[m h-1]"

        # exponent parameter (1.0 for linear reservoir)
        d["expon"] = "6.0[]"

        # initial condition for groundwater reservoir - it is the ground water as a
        # decimal fraction of the maximum groundwater storage (max_gw_storage) for the initial timestep
        d["gw_storage"] = "0.05[m/m]"

        # field capacity
        d["alpha_fc"] = "0.33[]"

        # initial condition for soil reservoir - it is the water in the soil as a
        # decimal fraction of maximum soil water storage (smcmax * depth) for the initial timestep
        d["soil_storage"] = "0.05[m/m]"

        # number of Nash lf reservoirs (optional, defaults to 2, ignored if storage values present)
        d["K_nash"] = "0.03[]"

        # Nash Config param - primary reservoir
        d["K_lf"] = "0.01[]"

        # Nash Config param - secondary reservoir
        d["nash_storage"] = "0.0,0.0"

        # Giuh ordinates in dt time steps
        d["giuh_ordinates"] = "1.00,0.00"

        # ---------------------
        # Time Info
        # ---------------------

        # set to 1 if forcing_file=BMI
        d["num_timesteps"] = "1"

        # ---------------------
        # Options
        # ---------------------

        # prints various debug and bmi info
        d["verbosity"] = "0"

        d["DEBUG"] = "0"

        # Parameter in the surface runoff parameterization
        # (https://mikejohnson51.github.io/hyAggregate/#Routing_Attributes)
        d["refkdt"] = f'{row["refkdt"]}'

        catchment_configs[row.divide_id] = d

    return catchment_configs


## Create CFE Config

In [5]:
noah_params_path = Path('../test-data/test-small/domain/cfe_noahowp_attributes.csv')
df_noah_params = pandas.read_csv(noah_params_path)
catchment_configs = parse_cfe_parameters(df_noah_params)

In [6]:
# work with a single catchment
cat_id = 'cat-2918021'

# load noah data for this catchment
cat_dict = catchment_configs[cat_id]

cat_dict

OrderedDict([('forcing_file', 'BMI'),
             ('surface_partitioning_scheme', 'Schaake'),
             ('soil_params.depth', '2.0[m]'),
             ('soil_params.b', '2.3282833099365234[]'),
             ('soil_params.satdk', '3.308438783921123e-07[m s-1]'),
             ('soil_params.satpsi', '0.0715057436219558[m]'),
             ('soil_params.slop', '0.3748068213462829[m/m]'),
             ('soil_params.smcmax', '0.5200302004814148[m/m]'),
             ('soil_params.wltsmc', '0.0469999983906745[m/m]'),
             ('soil_params.expon', '1.0[]'),
             ('soil_params.expon_secondary', '0.005[]'),
             ('max_gw_storage', '122.64931488[m]'),
             ('Cgw', '0.0018[m h-1]'),
             ('expon', '6.0[]'),
             ('gw_storage', '0.05[m/m]'),
             ('alpha_fc', '0.33[]'),
             ('soil_storage', '0.05[m/m]'),
             ('K_nash', '0.03[]'),
             ('K_lf', '0.01[]'),
             ('nash_storage', '0.0,0.0'),
             ('giuh_ordi

In [7]:
# create CFE object from these data
cfe_conf_base = cfe_init.CFEBase(
    **cat_dict
)

In [10]:
cfe_conf_base.Config.preserve_key_case

False

In [9]:
cfe_conf_base.json()

'{"forcing_file": "BMI", "soil_params_depth": "2.0[m]", "soil_params_b": "2.3282833099365234[]", "soil_params_satdk": "3.308438783921123e-07[m s-1]", "soil_params_satpsi": "0.0715057436219558[m]", "soil_params_slop": "0.3748068213462829[m/m]", "soil_params_smcmax": "0.5200302004814148[m/m]", "soil_params_wltsmc": "0.0469999983906745[m/m]", "refkdt": 2.613785982131958, "soil_params_expon": "1.0[]", "soil_params_expon_secondary": "0.005[]", "max_gw_storage": "122.64931488[m]", "cgw": "0.0018[m h-1]", "expon": "6.0[]", "gw_storage": "0.05[m/m]", "alpha_fc": "0.33[]", "soil_storage": "0.05[m/m]", "k_nash": "0.03[]", "k_lf": "0.01[]", "nash_storage": "0.0,0.0", "giuh_ordinates": "1.0,0.0", "num_timesteps": 1, "verbosity": 0, "surface_partitioning_scheme": "Schaake"}'

In [15]:
cfe_conf_base.Config.no_section_headers=True
cfe_conf_base.Config.space_around_delimiters=False
cfe_conf_base.Config.preserve_key_case=True
print(cfe_conf_base.to_ini_str())


forcing_file=BMI
soil_params.depth=2.0[m]
soil_params.b=2.3282833099365234[]
soil_params.satdk=3.308438783921123e-07[m s-1]
soil_params.satpsi=0.0715057436219558[m]
soil_params.slop=0.3748068213462829[m/m]
soil_params.smcmax=0.5200302004814148[m/m]
soil_params.wltsmc=0.0469999983906745[m/m]
refkdt=2.613785982131958
soil_params.expon=1.0[]
soil_params.expon_secondary=0.005[]
max_gw_storage=122.64931488[m]
Cgw=0.0018[m h-1]
expon=6.0[]
gw_storage=0.05[m/m]
alpha_fc=0.33[]
soil_storage=0.05[m/m]
K_nash=0.03[]
K_lf=0.01[]
nash_storage=0.0,0.0
giuh_ordinates=1.0,0.0
num_timesteps=1
verbosity=0
surface_partitioning_scheme=Schaake


In [11]:
print(cfe_conf_base.json(by_alias=True, indent=4))

{
    "forcing_file": "BMI",
    "soil_params.depth": "2.0[m]",
    "soil_params.b": "2.3282833099365234[]",
    "soil_params.satdk": "3.308438783921123e-07[m s-1]",
    "soil_params.satpsi": "0.0715057436219558[m]",
    "soil_params.slop": "0.3748068213462829[m/m]",
    "soil_params.smcmax": "0.5200302004814148[m/m]",
    "soil_params.wltsmc": "0.0469999983906745[m/m]",
    "refkdt": 2.613785982131958,
    "soil_params.expon": "1.0[]",
    "soil_params.expon_secondary": "0.005[]",
    "max_gw_storage": "122.64931488[m]",
    "Cgw": "0.0018[m h-1]",
    "expon": "6.0[]",
    "gw_storage": "0.05[m/m]",
    "alpha_fc": "0.33[]",
    "soil_storage": "0.05[m/m]",
    "K_nash": "0.03[]",
    "K_lf": "0.01[]",
    "nash_storage": "0.0,0.0",
    "giuh_ordinates": "1.0,0.0",
    "num_timesteps": 1,
    "verbosity": 0,
    "surface_partitioning_scheme": "Schaake"
}


In [13]:
print(cfe_conf_base.json(indent=4))

{
    "forcing_file": "BMI",
    "soil_params_depth": "2.0[m]",
    "soil_params_b": "2.3282833099365234[]",
    "soil_params_satdk": "3.308438783921123e-07[m s-1]",
    "soil_params_satpsi": "0.0715057436219558[m]",
    "soil_params_slop": "0.3748068213462829[m/m]",
    "soil_params_smcmax": "0.5200302004814148[m/m]",
    "soil_params_wltsmc": "0.0469999983906745[m/m]",
    "refkdt": 2.613785982131958,
    "soil_params_expon": "1.0[]",
    "soil_params_expon_secondary": "0.005[]",
    "max_gw_storage": "122.64931488[m]",
    "cgw": "0.0018[m h-1]",
    "expon": "6.0[]",
    "gw_storage": "0.05[m/m]",
    "alpha_fc": "0.33[]",
    "soil_storage": "0.05[m/m]",
    "k_nash": "0.03[]",
    "k_lf": "0.01[]",
    "nash_storage": "0.0,0.0",
    "giuh_ordinates": "1.0,0.0",
    "num_timesteps": 1,
    "verbosity": 0,
    "surface_partitioning_scheme": "Schaake"
}


## Create Formulations

### CFE

In [33]:
params = dict(name='bmi_c',
              init_config='test.ini', # replace
              library_file='/dmod/shared_libs/libcfebmi.so.1.0.0',
              allow_exceed_end_time = True,
              fixed_time_step = False,
              uses_forcing_file = True,
              forcing_file= '/ngen/data/forcing/cat-2918021.csv',  # replace
              variables_names_map = {"atmosphere_water__liquid_equivalent_precipitation_rate": "precip_rate",
                                     "water_potential_evaporation_flux": "EVAPOTRANS",
                                     "ice_fraction_schaake": "sloth_ice_fraction_schaake",
                                     "ice_fraction_xinan": "sloth_ice_fraction_xinan",
                                     "soil_moisture_profile": "sloth_smp"},
             )
              

In [34]:
c = cfe.CFE(**params)
f = {'params': c, 'name':'bmi_c'}
cfe_formulation = formulation.Formulation(**f)

In [35]:
print(cfe_formulation.json(by_alias=True, exclude_none=True, indent=4))

{
    "name": "bmi_c",
    "params": {
        "name": "bmi_c",
        "model_type_name": "CFE",
        "main_output_variable": "Q_OUT",
        "init_config": "test.ini",
        "allow_exceed_end_time": true,
        "fixed_time_step": false,
        "uses_forcing_file": true,
        "variables_names_map": {
            "atmosphere_water__liquid_equivalent_precipitation_rate": "precip_rate",
            "water_potential_evaporation_flux": "EVAPOTRANS",
            "ice_fraction_schaake": "sloth_ice_fraction_schaake",
            "ice_fraction_xinan": "sloth_ice_fraction_xinan",
            "soil_moisture_profile": "sloth_smp"
        },
        "library_file": "/dmod/shared_libs/libcfebmi.so.1.0.0",
        "registration_function": "register_bmi_cfe"
    }
}


### SLOTH

In [36]:
params = dict(name='bmi_c++',
              main_output_variable = "z",
              init_config = "/dev/null",
              allow_exceed_end_time = True,
              fixed_time_step = False,
              uses_forcing_file = False,
              model_params = {"sloth_ice_fraction_schaake(1,double,m,node)": "0.0",
                              "sloth_ice_fraction_xinan(1,double,1,node)": "0.0",
                              "sloth_smp(1,double,1,node)": "0.0",
                              "EVAPOTRANS": "0.0"},
              library_file = "/dmod/shared_libs/libslothmodel.so",
             )

In [37]:
s = sloth.SLOTH(**params)
f = {'params': s, 'name':'bmi_c++'}
sloth_formulation = formulation.Formulation(**f)

In [38]:
print(sloth_formulation.json(by_alias=True, exclude_none=True, indent=4))

{
    "name": "bmi_c++",
    "params": {
        "name": "bmi_c++",
        "model_type_name": "SLOTH",
        "main_output_variable": "z",
        "init_config": "/dev/null",
        "allow_exceed_end_time": true,
        "fixed_time_step": false,
        "uses_forcing_file": false,
        "model_params": {
            "sloth_ice_fraction_schaake(1,double,m,node)": "0.0",
            "sloth_ice_fraction_xinan(1,double,1,node)": "0.0",
            "sloth_smp(1,double,1,node)": "0.0",
            "EVAPOTRANS": "0.0"
        },
        "library_file": "/dmod/shared_libs/libslothmodel.so",
        "registration_function": "none"
    }
}


### BMI-MULTI

In [39]:
params = dict(name='bmi_multi',
              main_output_variable = "Q_OUT",
              init_config = "",
              allow_exceed_end_time = True,
              fixed_time_step = False,
              uses_forcing_file = False,
              modules = [sloth_formulation, cfe_formulation],
             )

In [40]:
m = multi.MultiBMI(**params)

In [41]:
f = {'params': m, 'name':m.name}
multi_formulation = formulation.Formulation(**f)

In [43]:
print(multi_formulation.json(by_alias=True, exclude_none=True, indent=4))

{
    "name": "bmi_multi",
    "params": {
        "name": "bmi_multi",
        "model_type_name": "SLOTH_CFE",
        "main_output_variable": "Q_OUT",
        "init_config": "",
        "allow_exceed_end_time": true,
        "fixed_time_step": false,
        "uses_forcing_file": false,
        "modules": [
            {
                "name": "bmi_c++",
                "params": {
                    "name": "bmi_c++",
                    "model_type_name": "SLOTH",
                    "main_output_variable": "z",
                    "init_config": "/dev/null",
                    "allow_exceed_end_time": true,
                    "fixed_time_step": false,
                    "uses_forcing_file": false,
                    "model_params": {
                        "sloth_ice_fraction_schaake(1,double,m,node)": "0.0",
                        "sloth_ice_fraction_xinan(1,double,1,node)": "0.0",
                        "sloth_smp(1,double,1,node)": "0.0",
                        "EVAPOT

### Forcing

In [20]:
forcing_path = Path('../extra/old stuff/working-example/forcings')
provider = configurations.Forcing.Provider.CSV
forcing = configurations.Forcing(file_pattern=".*{{id}}.*.csv", path=forcing_path, provider=provider)

In [21]:
forcing

Forcing(file_pattern='.*{{id}}.*.csv', path=PosixPath('../extra/old stuff/working-example/forcings'), provider=<Provider.CSV: 'CsvPerFeature'>)

### Realization

In [22]:
from ngen.config.realization import Realization

In [23]:
r = Realization(formulations=[multi_formulation], forcing=forcing)

In [24]:
print(r.json(indent=4))

{
    "formulations": [
        {
            "name": "bmi_multi",
            "params": {
                "name": "bmi_multi",
                "model_name": "SLOTH_CFE",
                "main_output_variable": "Q_OUT",
                "config": "",
                "allow_exceed_end_time": true,
                "fixed_time_step": false,
                "uses_forcing_file": false,
                "name_map": null,
                "output_vars": null,
                "output_headers": null,
                "model_params": null,
                "modules": [
                    {
                        "name": "bmi_c++",
                        "params": {
                            "name": "bmi_c++",
                            "model_name": "SLOTH",
                            "main_output_variable": "z",
                            "config": "/dev/null",
                            "allow_exceed_end_time": true,
                            "fixed_time_step": false,
                   

### NGEN Realization - BASIC

In [25]:
from ngen.config.configurations import Forcing, Time, Routing

In [26]:
t = Time(start_time="2019-06-01 00:00:00",
         end_time="2019-06-07 23:00:00",
         output_interval = 3600
        )

In [27]:
routing = Routing(t_route_config_file_with_path='/ngen/data/config/ngen.yaml')

In [28]:
routing

Routing(config=PosixPath('/ngen/data/config/ngen.yaml'), path='')

In [29]:
from ngen.config.realization import NgenRealization

In [30]:
ngen_realization = NgenRealization(global_config=r,
                                   time=t,
                                   routing=routing)

In [31]:
print(ngen_realization.json(indent=4))

{
    "global_config": {
        "formulations": [
            {
                "name": "bmi_multi",
                "params": {
                    "name": "bmi_multi",
                    "model_name": "SLOTH_CFE",
                    "main_output_variable": "Q_OUT",
                    "config": "",
                    "allow_exceed_end_time": true,
                    "fixed_time_step": false,
                    "uses_forcing_file": false,
                    "name_map": null,
                    "output_vars": null,
                    "output_headers": null,
                    "model_params": null,
                    "modules": [
                        {
                            "name": "bmi_c++",
                            "params": {
                                "name": "bmi_c++",
                                "model_name": "SLOTH",
                                "main_output_variable": "z",
                                "config": "/dev/null",
                 

### NGEN Realization - Catchments

In [32]:
from ngen.config.realization import CatchmentRealization

In [33]:
r = Realization(formulations=[multi_formulation], forcing=forcing)

In [34]:
cr = CatchmentRealization(**r.dict(by_alias=True))

In [35]:
provider = configurations.Forcing.Provider.CSV
global_forcing = configurations.Forcing(file_pattern="", path="", provider=provider)
global_realization = Realization(formulations=[], forcing=global_forcing)

In [36]:
ngen_realization = NgenRealization(global_config=global_realization,
                                   catchments={'cat-2918021':cr},
                                   time=t,
                                   routing=routing)

In [37]:
print(ngen_realization.json(indent=4))

{
    "global_config": {
        "formulations": [],
        "forcing": {
            "file_pattern": "",
            "path": ".",
            "provider": "CsvPerFeature"
        },
        "calibration": null
    },
    "time": {
        "start_time": "2019-06-01 00:00:00",
        "end_time": "2019-06-07 23:00:00",
        "output_interval": 3600
    },
    "routing": {
        "config": "/ngen/data/config/ngen.yaml",
        "path": ""
    },
    "catchments": {
        "cat-2918021": {
            "formulations": [
                {
                    "name": "bmi_multi",
                    "params": {
                        "name": "bmi_multi",
                        "model_name": "SLOTH_CFE",
                        "main_output_variable": "Q_OUT",
                        "config": "",
                        "allow_exceed_end_time": true,
                        "fixed_time_step": false,
                        "uses_forcing_file": false,
                        "name_map": 

## T-Route

In [11]:
# import sys
# sys.path.append(str(Path('../').resolve()))

In [12]:
# Installation
# git clone https://github.com/NOAA-OWP/t-route.git
# cd t-route/src/troute-config
# pip install .

In [16]:
from collections import OrderedDict
import pandas
import typing
from pathlib import Path

In [17]:

import troute.config as tconfig

In [18]:
from troute.config import config, \
                          logging_parameters, \
                          compute_parameters, \
                          bmi_parameters, \
                          network_topology_parameters, \
                          output_parameters

In [19]:
log_conf = logging_parameters.LoggingParameters(log_level='DEBUG',
                                                showtiming=True)
print(log_conf.json(indent=4))

{
    "log_level": "DEBUG",
    "showtiming": true
}


In [73]:
super_conf = network_topology_parameters.SupernetworkParameters(geo_file_path = '/ngen/data/domain/wb-2917533_upstream_subset.gpkg',
                                                                synthetic_wb_segments = None,
                                                                duplicate_wb_segments=None,
                                                               )
lp = network_topology_parameters.LevelPool(level_pool_waterbody_parameter_file_path='/ngen/data/domain/wb-2917533_upstream_subset.gpkg')
                                           
wb = network_topology_parameters.WaterbodyParameters(break_network_at_waterbodies=False,
                                                     level_pool=lp)
                                                     
network_conf = network_topology_parameters.NetworkTopologyParameters(supernetwork_parameters=super_conf,
                                                                     waterbody_parameters=wb)

## remove empty dictionaries
#cleaned = {k:v for k, v in network_conf.dict().items() if v}

print(network_conf.json(indent=4, by_alias=True, exclude_none=True))

{
    "preprocessing_parameters": {},
    "supernetwork_parameters": {
        "geo_file_path": "/ngen/data/domain/wb-2917533_upstream_subset.gpkg",
        "network_type": "HYFeaturesNetwork",
        "mask_layer_string": "",
        "mask_key": 0,
        "columns": {
            "key": "id",
            "downstream": "toid",
            "dx": "length_m",
            "n": "n",
            "ncc": "nCC",
            "s0": "So",
            "bw": "BtmWdth",
            "waterbody": "rl_NHDWaterbodyComID",
            "gages": "rl_gages",
            "tw": "TopWdth",
            "twcc": "TopWdthCC",
            "musk": "MusK",
            "musx": "MusX",
            "cs": "ChSlp",
            "alt": "alt",
            "mainstem": "mainstem"
        },
        "synthetic_wb_id_offset": 999000000000.0,
        "terminal_code": 0,
        "driver_string": "NetCDF",
        "layer_string": 0
    },
    "waterbody_parameters": {
        "break_network_at_waterbodies": false,
        "level_po

In [84]:
network_conf.supernetwork_parameters.columns

{'key': 'id',
 'downstream': 'toid',
 'dx': 'length_m',
 'n': 'n',
 'ncc': 'nCC',
 's0': 'So',
 'bw': 'BtmWdth',
 'waterbody': 'rl_NHDWaterbodyComID',
 'gages': 'rl_gages',
 'tw': 'TopWdth',
 'twcc': 'TopWdthCC',
 'musk': 'MusK',
 'musx': 'MusX',
 'cs': 'ChSlp',
 'alt': 'alt',
 'mainstem': 'mainstem'}

In [50]:
import datetime

In [54]:

st == datetime.datetime.timestamp

False

In [58]:
st.to_pydatetime()

datetime.datetime(2020, 1, 1, 0, 0)

In [60]:
# determine start and end dates from the forcing data
df = pandas.read_csv(list(Path('data/forcing').glob('*.csv'))[0])
st = pandas.to_datetime(df.time.min()).to_pydatetime()

# # set the end time 
# #et = pandas.to_datetime(df.time.max())
# from datetime import timedelta
# et = st + timedelta(days=10) # using 10 days to keep simulations short for now.

restart = compute_parameters.RestartParameters(
    start_datetime=st
)

hybrid = compute_parameters.HybridParameters(run_hybrid_routing=False)

forcing = compute_parameters.ForcingParameters(
    qts_subdivisions = 12,
    dt = 300,
    nts = 288 * 10,
    max_loop_size = 720,
    qlat_file_index_col = "feature_id",
    qlat_file_value_col = "q_lateral",
    qlat_file_gw_bucket_flux_col = "qBucket",
    qlat_file_terrain_runoff_col = "qSfcLatRunoff",
    qlat_file_pattern_filter = "nex-*",
    qlat_input_folder = str(Path('./')),
    binary_nexus_file_folder = str(Path('./')),
)

streamflow_da = compute_parameters.StreamflowDA(streamflow_nudging=False,
                                                diffusive_streamflow_nudging=False)
da = compute_parameters.DataAssimilationParameters(streamflow_da=streamflow_da)

comp_conf = compute_parameters.ComputeParameters(
    parallel_compute_method='by-subnetwork-jit-clustered',
    assume_short_ts = True,
    restart_parameters = restart,
    hybrid_parameters = hybrid,
    forcing_parameters = forcing,
    data_assimilation_parameters = da
)

print(comp_conf.json(indent=4, by_alias=True, exclude_none=True))

{
    "parallel_compute_method": "by-subnetwork-jit-clustered",
    "compute_kernel": "V02-structured",
    "assume_short_ts": true,
    "subnetwork_target_size": 10000,
    "cpu_pool": 1,
    "return_courant": false,
    "restart_parameters": {
        "start_datetime": "2020-01-01T00:00:00"
    },
    "hybrid_parameters": {
        "run_hybrid_routing": false,
        "use_natl_xsections": false,
        "run_refactored_network": false
    },
    "forcing_parameters": {
        "qts_subdivisions": 12,
        "dt": 300,
        "qlat_input_folder": ".",
        "nts": 2880,
        "max_loop_size": 720,
        "qlat_file_index_col": "feature_id",
        "qlat_file_value_col": "q_lateral",
        "qlat_file_gw_bucket_flux_col": "qBucket",
        "qlat_file_terrain_runoff_col": "qSfcLatRunoff",
        "qlat_file_pattern_filter": "nex-*",
        "binary_nexus_file_folder": "."
    },
    "data_assimilation_parameters": {
        "timeslice_lookback_hours": 24,
        "interpolati

In [61]:
## Output Parameters
"""
output_parameters:
    #----------
    #test_output: output/unit_test_hyfeature.pkl
    #lite_restart:
    #    #----------
    #    lite_restart_output_directory: restart/
  csv_output:
    csv_output_folder: ./
  chrtout_output:
        #----------
        #wrf_hydro_channel_output_source_folder: ./
    # chanobs_output:
    #     #----------
    #     chanobs_output_directory: ./routing
    #     chanobs_filepath        : Chanobs.nc
  lakeout_output: ./
"""

csv = output_parameters.CsvOutput(
    csv_output_folder = str(Path('./'))
)


out_conf = output_parameters.OutputParameters(
    csv_output = csv    
)

print(out_conf.json(indent=4, by_alias=True, exclude_none=True))

{
    "csv_output": {
        "csv_output_folder": "."
    }
}


In [62]:
conf = tconfig.Config(
    log_parameters = log_conf,
    compute_parameters = comp_conf,
    network_topology_parameters = network_conf,
    output_parameters = out_conf
)

## Validation

In [109]:
import troute.config 
from nwm_routing import input

ModuleNotFoundError: No module named 'troute.nhd_io'

In [108]:
dir(nwm_routing)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [91]:
import yaml
from yaml import Dumper, ScalarNode, add_representer
from pathlib import PosixPath
from datetime import datetime

def posixpath_handler(dumper: Dumper, data: PosixPath) -> ScalarNode:
    return dumper.represent_str(str(data))

def datetime_handler(dumper: Dumper, data: datetime) -> ScalarNode:
    return dumper.represent_str(data.strftime('%Y-%m-%d %H:%M:%S'))

add_representer(PosixPath, posixpath_handler)
add_representer(datetime, datetime_handler)
#print(yaml.dump(conf.dict(), default_flow_style = False, sort_keys=False))


with open('data/config/ngen.yaml', 'w') as outfile:
    yaml.dump(conf.dict(), outfile, default_flow_style = False, sort_keys=False)


In [92]:

with open('data/config/ngen.yaml') as custom_file:
    data = yaml.load(custom_file, Loader=yaml.SafeLoader)

troute_configuration = config.Config(**data)
config_dict = troute_configuration.dict()

In [96]:
log_parameters = config_dict.get('log_parameters')
compute_parameters = config_dict.get('compute_parameters')
network_topology_parameters = config_dict.get('network_topology_parameters')
output_parameters = config_dict.get('output_parameters')


preprocessing_parameters = network_topology_parameters.get('preprocessing_parameters')
supernetwork_parameters = network_topology_parameters.get('supernetwork_parameters')
waterbody_parameters = network_topology_parameters.get('waterbody_parameters')
forcing_parameters = compute_parameters.get('forcing_parameters')
restart_parameters = compute_parameters.get('restart_parameters')
hybrid_parameters = compute_parameters.get('hybrid_parameters')
parity_parameters = output_parameters.get('wrf_hydro_parity_check')
data_assimilation_parameters = compute_parameters.get('data_assimilation_parameters')




In [94]:
supernetwork_parameters

{'title_string': None,
 'geo_file_path': '/ngen/data/domain/wb-2917533_upstream_subset.gpkg',
 'network_type': 'HYFeaturesNetwork',
 'flowpath_edge_list': None,
 'mask_file_path': None,
 'mask_layer_string': '',
 'mask_driver_string': None,
 'mask_key': 0,
 'columns': {'key': 'id',
  'downstream': 'toid',
  'dx': 'length_m',
  'n': 'n',
  'ncc': 'nCC',
  's0': 'So',
  'bw': 'BtmWdth',
  'waterbody': 'rl_NHDWaterbodyComID',
  'tw': 'TopWdth',
  'twcc': 'TopWdthCC',
  'alt': 'alt',
  'musk': 'MusK',
  'musx': 'MusX',
  'cs': 'ChSlp',
  'gages': 'rl_gages',
  'mainstem': 'mainstem'},
 'synthetic_wb_segments': None,
 'synthetic_wb_id_offset': 999000000000.0,
 'duplicate_wb_segments': None,
 'terminal_code': 0,
 'driver_string': 'NetCDF',
 'layer_string': 0}

In [80]:
from pydantic_yaml import parse_yaml_raw_as, to_yaml_str

print(to_yaml_str(conf, by_alias=True, exclude_none=True))

compute_parameters:
  assume_short_ts: true
  compute_kernel: V02-structured
  cpu_pool: 1
  data_assimilation_parameters:
    interpolation_limit_min: 59
    qc_threshold: 1
    streamflow_da:
      crosswalk_gage_field: gages
      crosswalk_segID_field: link
      diffusive_streamflow_nudging: false
      streamflow_nudging: false
    timeslice_lookback_hours: 24
    wrf_hydro_lastobs_lead_time_relative_to_simulation_start_time: 0
    wrf_lastobs_type: obs-based
  forcing_parameters:
    binary_nexus_file_folder: .
    dt: 300
    max_loop_size: 720
    nts: 2880
    qlat_file_gw_bucket_flux_col: qBucket
    qlat_file_index_col: feature_id
    qlat_file_pattern_filter: nex-*
    qlat_file_terrain_runoff_col: qSfcLatRunoff
    qlat_file_value_col: q_lateral
    qlat_input_folder: .
    qts_subdivisions: 12
  hybrid_parameters:
    run_hybrid_routing: false
    run_refactored_network: false
    use_natl_xsections: false
  parallel_compute_method: by-subnetwork-jit-clustered
  restart_

In [37]:
import json
import yaml

# serialize to json
conf_json = conf.json(indent=4, by_alias=True, exclude_none=True)

# convert from json to yaml
yaml_str = yaml.dump(json.loads(conf_json), default_flow_style = False, sort_keys=False)

print(yaml_str)

log_parameters:
  log_level: DEBUG
  showtiming: true
network_topology_parameters:
  preprocessing_parameters: {}
  supernetwork_parameters:
    geo_file_path: /ngen/data/domain/wb-2917533_upstream_subset.gpkg
    network_type: HYFeaturesNetwork
    mask_layer_string: ''
    mask_key: 0
    columns:
      key: id
      downstream: toid
      dx: length_m
      n: n
      ncc: nCC
      s0: So
      bw: BtmWdth
      waterbody: rl_NHDWaterbodyComID
      gages: rl_gages
      tw: TopWdth
      twcc: TopWdthCC
      musk: MusK
      musx: MusX
      cs: ChSlp
      alt: alt
      mainstem: mainstem
    synthetic_wb_id_offset: 999000000000.0
    terminal_code: 0
    driver_string: NetCDF
    layer_string: 0
  waterbody_parameters:
    break_network_at_waterbodies: false
    level_pool:
      level_pool_waterbody_parameter_file_path: /ngen/data/domain/wb-2917533_upstream_subset.gpkg
      level_pool_waterbody_id: lake_id
    waterbody_null_code: -9999
compute_parameters:
  parallel_compute